In [55]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

module_path = '/home/minsoo/Workspace/RecSys-study/fastcampus_lecture/'
import sys
sys.path.append(module_path)
from fcRecSys.utils import cos_sim_matrix

In [56]:
path = '../data/movielens/'

ratings_df = pd.read_csv(path + 'ratings.csv')

In [57]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [58]:
num_user = len(ratings_df['userId'].unique())
num_movie = len(ratings_df['movieId'].unique())

print(f'user 수 = {num_user}')
print(f'movie 수 = {num_movie}')

user 수 = 610
movie 수 = 9724


In [59]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=0)

print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


# Item-based

In [67]:
item_user_sparse_matrix = train_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0) # 지금은 0으로 채웠는데 어떤 수로 채울지 상황에 따라 고민해봐야한다.

In [68]:
item_user_sparse_matrix.shape

(8975, 610)

In [69]:
item_user_sparse_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,0.0,2.5,4.0,2.5,3.0,0.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
cos_sim_matrix??

Signature:
cos_sim_matrix(
    a: pandas.core.frame.DataFrame,
    b: pandas.core.frame.DataFrame,
) -> pandas.core.frame.DataFrame
Source:   
def cos_sim_matrix(a: pd.DataFrame, b: pd.DataFrame) -> pd.DataFrame:
    """get cosine similarity matrix

    Parameters
    ----------
    a : pd.DataFrame
        _description_
    b : pd.DataFrame
        _description_

    Returns
    -------
    pd.DataFrame
        n * n cosine similarity matrix
    """
    sim_val = cosine_similarity(a.values, b.values)
    cos_sim_df = pd.DataFrame(data=sim_val, columns=a.index, index=a.index)
    return cos_sim_df
File:      ~/Workspace/RecSys-study/fastcampus_lecture/fcRecSys/utils.py
Type:      function


In [70]:
item_cos_sim_df = cos_sim_matrix(item_user_sparse_matrix, item_user_sparse_matrix)

print(item_cos_sim_df.shape)
item_cos_sim_df.head()

(8975, 8975)


movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.362034,0.257257,0.043655,0.267697,0.311795,0.252597,0.125792,0.197641,0.320940,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.362034,1.000000,0.296120,0.129730,0.249307,0.248522,0.182594,0.156356,0.042355,0.351247,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.257257,0.296120,1.000000,0.120192,0.369221,0.253139,0.261465,0.386738,0.136037,0.218627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.043655,0.129730,0.120192,1.000000,0.224904,0.106815,0.291399,0.189295,0.000000,0.079547,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.267697,0.249307,0.369221,0.224904,1.000000,0.259439,0.383786,0.321111,0.328702,0.162359,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 영화끼리의 유사도를 이용하여 평점예측해보기
    - `train_df`에서 유저마다 영화에 대한 평점들이 있고
    - 이 값을 `item_cos_sim_df`의 유사도를 이용하여 가중평균(vector 곱 연산)하여 예측

In [85]:
# 예측 결과 df
item_based_pred_df = pd.DataFrame(
    index=train_df['userId'].unique(),
    columns=item_cos_sim_df.index
    )

print(item_based_pred_df.shape)
item_based_pred_df.head()

(610, 8975)


movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193573,193579,193581,193583,193585,193587,193609
483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
train_user_group = train_df.groupby('userId')

for userId, group in tqdm(train_user_group):
    # train에서 유저가 평가한 영화들의 sim
    # |user_sim| = (# user rating movie, # total movie)
    user_sim = item_cos_sim_df.loc[group['movieId']]

    # 유저가 평가한 영화들의 평점
    # |user_rating| = (# user rating movie, )
    user_rating = group['rating']

    # normalize를 위한 sim sum
    sim_sum = np.sum(user_sim)

    # 예측
    # |pred_ratings| = (# total movie, )
    pred_ratings = np.matmul(user_sim.T.values, user_rating) / (sim_sum + 1)
    
    item_based_pred_df.loc[userId] = pred_ratings


100%|██████████| 610/610 [00:05<00:00, 117.75it/s]


In [88]:
print(item_based_pred_df.shape)
item_based_pred_df.head()

(610, 8975)


movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193573,193579,193581,193583,193585,193587,193609
483,3.602827,3.572719,3.449786,3.153353,3.514083,3.574495,3.479952,3.470818,2.905929,3.531961,...,2.823304,2.823304,2.823304,2.823304,2.823304,2.823304,2.823304,2.823304,2.823304,3.567246
599,2.7868,2.708775,2.646146,2.416319,2.627857,2.765293,2.622613,2.520901,2.408091,2.72597,...,2.820858,2.820858,2.820858,2.820858,2.820858,2.820858,2.820858,2.820858,2.820858,2.806938
247,3.708104,3.635426,3.524074,2.887203,3.483962,3.645656,3.480932,3.422085,2.559651,3.627657,...,2.398913,2.398913,2.398913,2.398913,2.398913,2.398913,2.398913,2.398913,2.398913,3.071712
111,3.259549,3.254555,3.208421,2.716086,3.416459,3.205431,3.202667,3.130531,2.228999,3.223671,...,3.28757,3.28757,3.28757,3.28757,3.28757,3.28757,3.28757,3.28757,3.28757,2.732805
610,3.800125,3.714296,3.718936,3.48592,3.713403,3.841964,3.777033,3.55455,3.343882,3.790562,...,3.451351,3.451351,3.451351,3.451351,3.451351,3.451351,3.451351,3.451351,3.451351,3.812632


# User-based

In [90]:
user_item_sparse_matrix = item_user_sparse_matrix.T

print(user_item_sparse_matrix.shape)
user_item_sparse_matrix.head()

(610, 8975)


movieId,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193567,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [91]:
user_cos_sim_df = cos_sim_matrix(user_item_sparse_matrix, user_item_sparse_matrix)

print(user_cos_sim_df.shape)
user_cos_sim_df.head()

(610, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.033270,0.002385,0.139888,0.095641,0.108416,0.125267,0.071205,0.053257,0.017417,...,0.054738,0.082883,0.177481,0.049590,0.120273,0.130904,0.185762,0.224306,0.064512,0.108721
2,0.033270,1.000000,0.000000,0.004483,0.020220,0.017163,0.033377,0.031425,0.000000,0.051374,...,0.210500,0.020693,0.014280,0.000000,0.000000,0.033618,0.015030,0.043176,0.033656,0.065164
3,0.002385,0.000000,1.000000,0.000000,0.000000,0.001476,0.000000,0.000000,0.000000,0.000000,...,0.006164,0.002373,0.015546,0.000000,0.000000,0.006908,0.021837,0.018038,0.000000,0.022469
4,0.139888,0.004483,0.000000,1.000000,0.066516,0.059636,0.082883,0.048243,0.015433,0.010827,...,0.038307,0.094737,0.235672,0.029699,0.071804,0.163998,0.082408,0.110289,0.023066,0.077550
5,0.095641,0.020220,0.000000,0.066516,1.000000,0.221221,0.055291,0.332205,0.000000,0.025011,...,0.038763,0.323658,0.078552,0.206621,0.146649,0.061130,0.091989,0.114280,0.213264,0.048756


In [92]:
# 예측 결과 df
user_based_pred_df = pd.DataFrame(
    index=train_df['movieId'].unique(),
    columns=user_cos_sim_df.index
    )

print(user_based_pred_df.shape)
user_based_pred_df.head()

(8975, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
8529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33437,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
train_movie_group = train_df.groupby('movieId')

for movieId, group in tqdm(train_movie_group):
    movie_sim = user_cos_sim_df.loc[group['userId']]
    user_rating = group['rating']
    sim_sum = np.sum(movie_sim)
    pred_ratings = np.matmul(movie_sim.T.values, user_rating) / (sim_sum + 1)
    user_based_pred_df.loc[movieId] = pred_ratings

100%|██████████| 8975/8975 [00:15<00:00, 594.89it/s]


In [94]:
print(user_based_pred_df.shape)
user_based_pred_df.head()

(8975, 610)


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
8529,2.775068,2.664977,0.66985,2.5688,2.538222,2.49131,3.02704,2.744187,2.351009,2.994617,...,3.065704,2.679376,2.664624,2.400307,2.885636,2.966562,2.745536,3.02854,2.344903,3.084748
33437,1.981132,0.988087,0.337997,1.550691,1.434323,1.664282,2.061856,1.567297,1.077171,1.368434,...,1.731076,1.759707,1.924965,1.469863,1.730629,2.067495,1.922797,2.589725,1.272928,2.223522
5349,3.340438,3.037413,1.480539,3.220621,3.113798,3.175625,3.401462,3.235018,3.137649,3.193864,...,3.302931,3.247008,3.290324,3.049438,3.333186,3.378648,3.331552,3.438661,3.100398,3.428258
7361,3.863375,3.677588,1.593371,3.73781,3.676565,3.60998,3.877922,3.729265,3.488413,3.661457,...,3.981244,3.759341,3.836454,3.559266,3.780136,3.942289,3.821731,4.002348,3.55605,3.990633
57504,2.143935,2.137489,0.285375,1.677196,1.475753,1.217536,2.410741,1.798735,1.47984,2.132825,...,2.825793,1.671533,1.911144,1.328442,2.31975,2.459954,1.89071,2.781256,1.166848,3.163699


# 추천, 성능평가